### 데이터의 이해 
- 변수의 평균, 표준편차, 최댓값, 최솟값, 분위수
- 범주형 변수의 종류 수 
- 변수의 결측값 수 
- 변수 간 상관관계 수 

------

- 시각화 기법
    - 막대그래프
    - 상자그림, 바이올린 플롯
    - 산점도
    - 꺾은선 그래프
    - 히트맵
    - 히스토그램
    - Q-Q플롯
    - t-SNE, UMAP

In [1]:
import pandas as pd 

In [16]:
train = pd.read_csv('../else/train.csv') #데이터 불러오기
test = pd.read_csv('../else/test.csv')

train_x = train.drop(['Survived'],axis=1) # data와 target 분리
train_y = train['Survived']

test_x = test.copy()

In [17]:
train_x.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [18]:
train_x = train_x.drop(["PassengerId"],axis=1)
test_x = test_x.drop(["PassengerId"],axis=1)
train_x = train_x.drop(["Name","Ticket","Cabin"],axis=1)
test_x = test_x.drop(["Name","Ticket","Cabin"],axis=1) # 필요없는 데이터 제거

for i in ['Sex',"Embarked"]: # 학습데이터,테스트 데이터 라벨 인코딩 
    le = LabelEncoder()
    train_x[i] = le.fit_transform(train_x[i].fillna("NA"))
    test_x[i] = le.transform(test_x[i].fillna('NA'))

In [33]:
train_x.head(3 )

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,3
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,3


In [52]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=20, random_state=71) # 모델에 학습
model.fit(train_x, train_y)

pred = model.predict_proba(test_x)[:,1] #테스트 데이터 예측결과를 확률로 출력 

import numpy as np 
pred_label = np.where(pred>0.5, 1, 0) # 0.5보다 큰수는 1로 아니면 0으로 반환 

submission = pd.DataFrame({'PassengerId': test["PassengerId"], 'survived':pred_label}) # 테스트 예측데이터 저장 
submission.to_csv('submission_first.csv', index=False) #인덱스 같이 내보내기 싫으면 False 

In [55]:
# 모델 검증해보기
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

In [56]:
# fold 평가점수를 위한 빈 리스트 생성
scores_accuracy = []
scores_logloss = []


In [61]:
# 교차 검증 수행
# 1. 학습데이터를 4개로 분할
# 2. 그중 하나를 평가용 데이터셋으로 지정
# 3. 이후 평가용 데이터의 블록을 하나씩 옆으로 옮겨가며 검증을 수행 
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 학습데이터를 학습데이터와 평가용 데이터셋으로 분할
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [76]:
# 모델 학습 수행
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(tr_x,tr_y)
# 평가용 데이터의 예측결과를 확률로 출력 
va_pred = model.predict_proba(va_x)[:,1]
# 평가용 데이터의 점수를 계산
logloss = log_loss(va_y, va_pred)
accuracy = accuracy_score(va_y, va_pred >0.5)
# 각 fold의 점수를 저장 
scores_logloss.append(logloss)
scores_accuracy.append(accuracy)
# 각 fold의 점수 평균을 출력 
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f"logloss: {logloss:.3f}, accuracy : {accuracy:.3f}") # f는 뒤에 소숫점 자리수 의미

logloss: 0.438, accuracy : 0.829


In [80]:
# 모델 튜닝 

import itertools

param_space = {
    'max_depth' : [3,5,7], # 트리의 최대 깊이입니다. None이면 모든 잎이 순수하거나 모든 잎이 min_samples_split 샘플 미만을 포함할 때까지 노드가 확장됩니다.
    'min_child_weight' : [1.0, 2.0, 4.0] # 자식에 필요한 인스턴스 가중치(헤시안)의 최소 합입니다, 클수록 min_child_weight알고리즘이 더 보수적입니다.
}
# 하이퍼 파라미터의 값을 조합
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight']) # 두개 이상의 리스트(or 집합) 끼리의 데카르트 곱(cartesian product)를 계산하여 iterator로 반환해준다.

# 각 파라미터의 조합과 그에 대한 점수를 보존하는 빈 리스트
params = []
scores = []

In [81]:
for max_depth, min_child_weight in param_combinations:
    score_folds = []
# 교차 검증 수행
# 학습데이터 4개로 분할
# 그중 하나를 평가용 데이터로 삼아서 평가 
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 학습데이터를 학습데이터와 평가용 데이터셋으로 분할
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        #모델의 학습을 수행 
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth,
                              min_child_weight=min_child_weight)
        model.fit(tr_x,tr_y)
        #검증용 데이터의 점수 계산 후 저장 
        va_pred = model.predict_proba(va_x)[:,1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
    # 각 fold의 점수 평균을 구함 
    score_mean = np.mean(score_folds)
    # 파라미터를 조합하고 그에 대한 점수를 저장 
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)
# 가장 점수가 좋은것을 베스트 파라미터로 지정 
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth:{best_param[0]}, min_child_weight: {best_param[1]}')

max_depth:3, min_child_weight: 2.0
